# Setup


In [ ]:
# 1. Imports e setup
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from opacus import PrivacyEngine

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# If you desire reproducibility, run the following function:

def set_seed(seed=42):
    """
    Sets the random seed for reproducibility across various runs.
    """
    import os
    import random
    import numpy as np

    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True # Makes training slower but ensures reproducibility
    torch.backends.cudnn.benchmark = False

set_seed()

In [ ]:
from tqdm import tqdm

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def evaluate_model(model, dataloader):
    """
    Avalia o modelo em um conjunto de dados e retorna as métricas de acurácia e F1-score
    """
    # Certifique-se de que o modelo esteja em modo de avaliação
    model.eval()

    # Coletar todas as previsões e rótulos reais
    y_true = []
    y_pred = []

    with torch.no_grad():
        for x, y, att_mask in tqdm(dataloader, desc="epoch"):
            x, y, att_mask = x.to(device), y.to(device), att_mask.to(device)
            logits = model(x, attention_mask=att_mask)
            preds = torch.argmax(logits, dim=1)
            y_true.extend(y.cpu().tolist())
            y_pred.extend(preds.cpu().tolist())

    # Calcular métricas
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')  # usa média ponderada por classe

    print(f"Acurácia: {acc:.4f}")
    print(f"F1-score (weighted): {f1:.4f}")


In [ ]:
BATCH_SIZE = 128
MAX_PHYSICAL_BATCH_SIZE = 128

In [ ]:
from dataset  import MovieDataset  # Certifique-se de que o arquivo dataset.py está no mesmo diretório

train_dataset = MovieDataset(train=True)
test_dataset = MovieDataset(train=False)

num_labels = train_dataset.num_labels # Get num_labels from dataset

# Instantiate DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Model Definition

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from models.madlib import MadlibModel
model = MadlibModel(2,epsilon=5).to(device).train()

In [ ]:
criterion = CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

num_epochs = 3
total_steps = len(train_loader) * num_epochs
warmup_steps = int(0.1 * total_steps)


In [ ]:
def evaluate(model, dataloader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for x, y, att_mask in tqdm(dataloader, desc="epoch"):
            x, y, att_mask = x.to(device), y.to(device), att_mask.to(device)
            logits = model(x, attention_mask=att_mask)
            preds = torch.argmax(logits, dim=1)

            y_true.extend(y.cpu().tolist())
            y_pred.extend(preds.cpu().tolist())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"Validation Accuracy: {acc:.4f}, F1-Score: {f1:.4f}")
    return acc, f1


In [ ]:
for epoch in range(3):
    model.train()
    total_loss = 0
    for step, (x, y, att_mask) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
        x, y, att_mask = x.to(device), y.to(device), att_mask.to(device)

        optimizer.zero_grad()
        logits = model(x, attention_mask=att_mask)
        loss = criterion(logits, y.long())
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f}")

    evaluate(model, test_loader)


In [ ]:
# 3. Save the state_dict
PATH = "model_weights.pth"
torch.save(model.state_dict(), PATH)
print(f"Model weights saved to {PATH}")

# DP-SGD

In [ ]:
EPOCHS = 3
LOGGING_INTERVAL = 5000 # once every how many steps we run evaluation cycle and report metrics
EPSILON = 7.5
DELTA = 1 / len(train_loader) # Parameter for privacy accounting. Probability of not achieving privacy guarantees

In [ ]:
from opacus import PrivacyEngine

MAX_GRAD_NORM = 0.1

privacy_engine = PrivacyEngine()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, eps=1e-8)
criterion = CrossEntropyLoss()

model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    target_delta=DELTA,
    target_epsilon=EPSILON,
    epochs=EPOCHS,
    max_grad_norm=MAX_GRAD_NORM,
)

In [ ]:
from opacus.utils.batch_memory_manager import BatchMemoryManager

for epoch in range(1, EPOCHS+1):
    model.train()
    losses = []

    with BatchMemoryManager(
        data_loader=train_dataloader,
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE,
        optimizer=optimizer
    ) as memory_safe_data_loader:
        for step, (x, y, att_mask) in enumerate(tqdm(memory_safe_data_loader)):
            x, y, att_mask = x.to(device), y.to(device), att_mask.to(device)
            optimizer.zero_grad()
            logits = model(x, attention_mask=att_mask)
            loss = criterion(logits, y.long())
            loss.backward()
            optimizer.step()

In [ ]:
# 6. Cálculo do epsilon após o treino E avaliação do modelo
epsilon = privacy_engine.accountant.get_epsilon(delta=1e-5)
print(f"ε = {epsilon:.2f}, δ = 1e-5")
evaluate_model(model, dataloader)

# Baseline

In [ ]:
# --- BASELINE SEM PRIVACIDADE ---
from torch.optim import Adam

model_baseline = TextClassifier(BaselineModel(),embedding_dim, num_classes).to(device)
optimizer = Adam(model_baseline.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

model_baseline.train()
for epoch in range(3):
    total_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model_baseline(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[BASELINE] Epoch {epoch+1} - Loss: {total_loss:.4f}")


In [ ]:
evaluate_model(model_baseline, dataloader)

# Embedding Noise

In [ ]:
def add_noise_to_embeddings(embedding_layer, sigma=0.1):
    with torch.no_grad():
        noise = torch.randn_like(embedding_layer.weight) * sigma
        embedding_layer.weight.add_(noise)


In [ ]:
# --- EMBEDDING PERTURBATION ---
model_embed = TextClassifier(BaselineModel(),embedding_dim, num_classes).to(device)

# Aplica ruído antes do treino
add_noise_to_embeddings(model_embed.embedding, sigma=0.1)

optimizer = Adam(model_embed.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

model_embed.train()
for epoch in range(3):
    total_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model_embed(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[EMBED NOISE] Epoch {epoch+1} - Loss: {total_loss:.4f}")


In [ ]:
evaluate_model(model_embed, dataloader)

In [ ]:
import numpy as np

# Parâmetros
sigma = 0.1          # mesmo que você usou ao aplicar ruído
delta = 1e-5
sensitivity = 1.0    # padrão

# Fórmula para mecanismo Gaussiano (epsilon aproximado)
epsilon = (np.sqrt(2 * np.log(1.25 / delta)) * sensitivity) / sigma

# Se aplicou ruído T vezes (ex: por época), multiplique:
T = 3  # ou 3, se adicionou ruído por época
epsilon_total = epsilon * T

print(f"ε ≈ {epsilon_total:.4f} (para σ = {sigma}, δ = {delta}, T = {T})")


# TEM

In [ ]:
from models.tem import TEMModel
model_tem = TextClassifier(TEMModel(),embedding_dim, num_classes).to(device)

In [ ]:
def apply_tem_noise(embedding_layer: torch.nn.Embedding, sigma: float = 0.1):
    """
    Aplica ruído gaussiano diretamente na camada de embeddings.
    """
    with torch.no_grad():
        noise = torch.randn_like(embedding_layer.weight) * sigma
        embedding_layer.weight.add_(noise)

# --- Aplicar TEM antes do treino ---
sigma_tem = 1.0  # ajuste para obter melhor privacidade (ε ↓)
apply_tem_noise(model_tem.embedding, sigma=sigma_tem)
print(f"TEM aplicado com sigma = {sigma_tem}")


In [ ]:
optimizer = torch.optim.Adam(model_tem.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

model_tem.train()
for epoch in range(3):
    total_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model_tem(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[TEM] Epoch {epoch+1} - Loss: {total_loss:.4f}")


In [ ]:
evaluate_model(model_tem, dataloader)

# MADLIB - TODO

In [ ]:
from models.madlib import MadlibModel

model = TextClassifier(MadlibModel(epsilon=5),embedding_dim, num_classes).to(device)
model.to(device)
model.eval()

# DP-SGD + TEM

In [ ]:
# Instancia o modelo
model_combo = TextClassifier(TEMModel(),embedding_dim, num_classes).to(device)

# --- Aplica ruído TEM às embeddings ---
sigma_tem = 1.0
def apply_tem_noise(embedding_layer: torch.nn.Embedding, sigma: float = 0.1):
    with torch.no_grad():
        noise = torch.randn_like(embedding_layer.weight) * sigma
        embedding_layer.weight.add_(noise)

apply_tem_noise(model_combo.embedding, sigma=sigma_tem)
print(f"[COMBO] TEM aplicado com σ = {sigma_tem}")


In [ ]:
optimizer = torch.optim.Adam(model_combo.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# Configura o PrivacyEngine
privacy_engine = PrivacyEngine()
model_combo, optimizer, dataloader_combo = privacy_engine.make_private(
    module=model_combo,
    optimizer=optimizer,
    data_loader=dataloader,
    noise_multiplier=1.0,      # σ do DP-SGD
    max_grad_norm=1.0,
)

# Treinamento
model_combo.train()
for epoch in range(3):
    total_loss = 0
    for x, y in dataloader_combo:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model_combo(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[COMBO] Epoch {epoch+1} - Loss: {total_loss:.4f}")


In [ ]:
epsilon_combo = privacy_engine.accountant.get_epsilon(delta=1e-5)
print(f"[COMBO] ε (DP-SGD): {epsilon_combo:.2f} | δ = 1e-5")
print(f"[COMBO] TEM aplicado com σ = {sigma_tem} (ε estimado separadamente ≈ {4.84 if sigma_tem==1.0 else 'recalcular'})")
evaluate_model(model_combo, dataloader)